# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from wordcloud import WordCloud, STOPWORDS

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [103]:
data = pd.read_excel("OneDrive/Projeto2cdd/mcdonalds.xlsx",sheet_name = "Treinamento")
colunas = ["Treinamento", "Classificação"]
data = data.loc[:,colunas]


#Limpando caracteres irrelevantes:

def deletar_caracteres(coluna):
    coluna = coluna.replace("."," ").replace(":"," ").replace(","," ").replace("'","").replace('"', " ").replace("rt"," ")
    return coluna

    

data['Treinamento'] = data['Treinamento'].apply(deletar_caracteres)
data.head(50)


,Treinamento,Classificação
0,@bbcbrasil jovens colam pôster publicitário...,0
1,moço no metrô com um saco do mcdonalds e agor...,2
2,@awannable younghoon tentando falar mcdona...,0
3,@joshhduh essa hora o mcdonalds ainda não f...,2
4,@mais_beea era só um lanche do mcdonalds pr...,2
5,@awannable younghoon tentando falar mcdona...,0
6,@mcdonalds_br sete dias sete opo unidades ...,2
7,@bbcbrasil estudantes de origem asiática fi...,0
8,nossa queria comprar todas as batatas do mcdon...,2
9,pelo menos vou passar no mcdonalds de novo kkkk,2


In [104]:
#Dividindo entre pos, neg e irrelevante
pos = data[data["Classificação"] == 2]
neg = data[data["Classificação"] == 1]
irrel = data[data["Classificação"] == 0] 
pos
    

,Treinamento,Classificação
1,moço no metrô com um saco do mcdonalds e agor...,2
3,@joshhduh essa hora o mcdonalds ainda não f...,2
4,@mais_beea era só um lanche do mcdonalds pr...,2
6,@mcdonalds_br sete dias sete opo unidades ...,2
8,nossa queria comprar todas as batatas do mcdon...,2
9,pelo menos vou passar no mcdonalds de novo kkkk,2
11,meu deus do céu alguém quer me comprar um ched...,2
13,processo pré-cisos \nalmoçar mcdonald’s compr...,2
18,@izafaria2018 ganhei um lanche do mcdonalds...,2
22,meu namorado passou no mcdonald’s dps do curso...,2


In [105]:

def word_count(Treinamento, word):
    cont = 0
    Treinamento = Treinamento.lower()
    if word in Treinamento.lower():
        return Treinamento.count(word)
    else :
        return 0 
data["batatas"] = data["Treinamento"].apply(lambda x: word_count(x, "batatas")) 
data["burguer king"] = data["Treinamento"].apply(lambda x: word_count(x, "burguer king")) 
data["fome"] = data["Treinamento"].apply(lambda x: word_count(x, "fome")) 
x = data.head(500)
frequencia = data.groupby("Classificação")[x.columns].sum()


frequencia








,Classificação,batatas,burguer king,fome
Classificação,,,,
0,0,0,0,2
1,60,0,0,0
2,234,3,0,2


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
